### 파라미터 수 구하기

In [2]:
# 파라미터 수 구하기
num = sum([p.numel() for p in model.parameters() if p.requires_grad])
print(num)

NameError: ignored

### 모델을 깊게 만들 때

In [3]:
# 모델을 깊게 만들 때
import torch
from torch import nn

nn.Sequential(*[nn.Linear(100, 100) if i % 2 == 0 else nn.Sigmoid() for i in range(10)])
nn.Sequential(*[i for _ in range(5) for i in [nn.Linear(100,100), nn.Sigmoid()]])

# 위와 같은 리스트 표현식이 중첩되어 있으면 순서대로 사용하면 된다.
a = []
for _ in range(5):
    for i in [nn.Linear(100,100), nn.Sigmoid()]:
        a. append(i)


# module_list=[]
# module_list.append(layer)
# nn.Sequential(*moudel_list) -> list를 unzip해서 사용

### 모델 학습, 평가 디폴트 포맷

In [4]:
# 모델 학습 디폴트 포맷
from torch import optim
optimizer = optim.SGD(model.parameters(), lr = LR)
# optimizer가 weight를 가지고 있다. 여기서 backward를 다 진행한다.
criterion= nn.CrossEntropyLoss

model.train()
for ep in range(EPOCH):
    for x_batch, y_batch in train_DL:
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        #inference
        y_hat = model(x)
        #loss
        loss = criterion(y_hat, y)
        #update
        optimizer.zero_grad() # 누적되는 그레디언트를 초기화
        loss.backward() #y_hat은 모델이 가지고 있는 파라미터를 다 품고 있다. 즉, leaf텐서에 대해 backward를 진행할 수 있음.
        optimizer.step() #optimizer에 weight가 있기 때문에 위에서 backward로 구해진 기울기 값을 이용해서 웨이트 업데이트를 진행한다.

NameError: ignored

In [ ]:
# 모델 테스트 디폴트 포맷
model.eval()
with torch.no_grad():
    rcorrect = 0
    for x_batch, y_batch in test_DL:
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        # inference
        y_hat = model(x_batch)
        # accuracy accumulation
        pred = y_hat.argmax(dim=1)
        corrects_b = torch.sum(pred==y_batch).item()
        rcorrect += corrects_b
    accuracy_e = rcorrect/len(test_DL.dataset) * 100
print(f"Test accuracy: {rcorrect}/{len(test_DL.dataset)} ({round(accuracy_e,1)} %)")

### 모델 저장, dataloader 데이터 불러오기
### 문자열을 코드로 바꿔서 요구되는 모델 불러오기

In [ ]:
# 모델 저장
save_model_path = '/content/drive/MyDrive/MLP_MNIST.pt'
torch.save(model.state_dict(), save_model_path)
model.state_dict()

load_model = MLP().to(DEVICE) # 껍데기에 알맹이를 넣는 방식
load_model.load_state_dict(torch.load(save_model_path), map_location=DEVICE) # map_location을 해줘야 cpu에서도 돌아감

In [ ]:
# Dataloader 데이터 불러오기
x_batch, y_batch = next(iter(train_DL)) # 데이터 한 국자

In [ ]:
# 문자열을 코드로 바꿔서 요구되는 모델을 불러오기
exec(f"model = {model_type}().to(DEVICE)")

In [ ]:
x=torch.randn(2,5)
layer=nn.ReLU()
print(layer(x)) # 음수값이 0이되는걸 확인할 수 있다.

In [ ]:
x = torch.randn(3,7)
drop = nn.Dropout(p=0.3) # p는 죽일 확률
print(drop(x)) # 죽은건 0이 되는걸 확인할 수 있다.

In [ ]:
# trasform

# # ToTensor의 역할
# 1. tensor로 바꿔준다.
# 2. 개채행열로 바꿔준다
# 3. 0~1 사이로 바꿔준다 (int -> float)

### Custom Dataset

In [ ]:
# Custom Dataset
import torch
from torchvision import transforms
import numpy as np

class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = self.X[idx]
        if self.transform is not None:
            self.transform(x)
        y = self.Y[idx]
        return x, y

In [ ]:
X_data = np.arange(-10,10).reshape(-1,1)
print(X_data.shape)
Y_data = X_data**2

In [ ]:
transform = lambda x:x+1
BATCH_SIZE = 8

custom_DS = Custom_Dataset(X_data, Y_data, transform=transform)

NoT = int(len(custom_DS)*0.8)
NoV = int(len(custom_DS)*0.1)
NoTes = len(custom_DS) - NoT - NoV
train_DS, val_DS, test_DS = torch.utils.data.random_split(custom_DS, [NoT, NoV, NoTes])

train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
val_DL = torch.utils.data.DataLoader(val_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
for x_batch, y_batch in train_DL:
    print(f"x_batch = {x_batch.reshape(-1)}, \n"
          f"y_batch = {y_batch.reshape(-1)}")
print()
for x_batch, y_batch in val_DL:
    print(f"x_batch = {x_batch.reshape(-1)}, \n"
          f"y_batch = {y_batch.reshape(-1)}")
print()
for x_batch, y_batch in test_DL:
    print(f"x_batch = {x_batch.reshape(-1)}, \n"
          f"y_batch = {y_batch.reshape(-1)}")

### .parameters() vs .modules() vs .children() 그리고 isinstance의 활용

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Sequential(nn.Linear(2,3),
                                 nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(3,4),
                                 nn.ReLU())
        self.fc_out = nn.Sequential(nn.Linear(4,1),
                                    nn.Sigmoid())
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc_out(x)
        return x

model = MLP()
print(model(torch.randn(2,2)).shape)
print(model)

In [ ]:
model.parameters()

In [ ]:
list(model.parameters())[0]
# [layer0 weight 값, layer0 bias 값, layer1 weight 값, layer1 bias 값, ...]

In [ ]:
# for tranfer learning
model=MLP()
print([p for p in model.parameters() if p.requires_grad])

for p in model.parameters(): # 전체 freeze
    p.requires_grad=False
model.fc_out = nn.Linear(4,10)

params = [p for p in model.parameters() if p.requires_grad]
print(params)

from torch import optim
optimizer = optim.Adam(params, lr=0.1)

In [ ]:
list(model.named_parameters())[0]
# [('layer0.weight', weight 값), ('layer0.bias', bias 값), ('layer1.weight', weight 값), ('layer1.bias', bias 값), ...]

In [ ]:
for name, p in model.named_parameters():
    print(name)
    print(p)

In [ ]:
model.modules()

In [ ]:
list(model.modules())

In [ ]:
# modules는 전체에서 개별로 하나하나 다 접근하기 때문에 isinstance를 이용해서 특정한 레이어를 선택할 수 있다.
print([m for m in model.modules() if isinstance(m,nn.Linear)])
print([m.weight for m in model.modules() if isinstance(m,nn.Linear)])
print([m.weight.grad for m in model.modules() if isinstance(m,nn.Linear)])

In [ ]:
# weight initialization에 활용
for m in model.modules():
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        # nn.init.constant_(m.weight, 1)

print([m.weight for m in model.modules() if isinstance(m, nn.Linear)])

In [ ]:
model.children()

In [ ]:
list(model.children())

In [ ]:
# 모델에서 특정한 위치에 있는 레이어에서 foward를 할 때 사용
x = torch.randn(2,2)
print(list(model.children())[0])
list(model.children())[0](x)

In [ ]:
# sub_network를 만들때에도 사용
print(*list(model.children())[:2])
sub_network = nn.Sequential(*list(model.children())[:2])
print(sub_network)
print(sub_network(x))

### ModuleList vs Sequential

In [ ]:
fc=nn.Linear(3,3)
layer_list = [fc for _ in range(5)]
layers1 = nn.Sequential(*layer_list)
layers2 = nn.ModuleList(layer_list)
print(layers1) # Sequential은 * unzip을 이용해야한다.
print(layers2) # ModuleList는 리스트를 그냥 줘도 된다.

x=torch.randn(1,3)
print(layers1(x)) # 잘됨! -> forward(self,x)가 정의되어 있다.

# print(layers2(x)) # error! -> forward(self,x)가 정의되어 있지 않다.
for layer in layers2: # 리스트에 들어 있는 layer를 하나씩 불러와서 사용하면 된다.
    x = layer(x)
print(x)

In [ ]:
# 걍 list 쓰지 왜 nn.ModuleList 를 쓸까?
class testNet(nn.Module):
    def __init__(self):
        super().__init__()

        # self.Module_List = [nn.Linear(3,3), nn.Linear(3,3)] # 걍 list를 쓰면 모델에 등록이 안된다.
        self.Module_List = nn.ModuleList([nn.Linear(3,3), nn.Linear(3,3)])

    def forward(self,x):
        for layer in self.Module_List:
            x = layer(x)
        return x

model=testNet()
print(model(torch.randn(1,3)))

print(model) # 그냥 리스트로 하면 등록이 안돼있다!
print(model.parameters())

optimizer = optim.Adam(model.parameters(), lr = 0.1) # 등록이 안돼있으면 parameter를 못 찾는다!

In [ ]:
# 그럼 nn.Sequential 쓰고 말지 왜 굳이 nn.ModuleList?
class small_block(nn.Module):
    def __init__(self):
        super().__init__()
        self.block_x = nn.Linear(1,1)
        self.block_y = nn.Linear(1,1)

    def forward(self, x, y):
        x = self.block_x(x)
        y = self.block_y(y)
        return x, y

block = small_block()
print(block)
model = nn.Sequential(block, block)
print(model)
# model(torch.randn(1), torch.randn(1)) # error!
# nn.Sequential 이 가지고 있는 forward 함수를 call 하는데, 해당 함수의 파라미터는 x 하나의 변수만 받는다.
# 따라서 위와 같은 상황에선 x, y 2개의 변수를 가지기 때문에 에러가 뜬다.
#    -> block에서의 foward는 잘 통과된다. 하지만 block과 block을 Sequential로 풀칠한 그 부분에서 오류가 뜸.
# 이를 해결하기 위해 ModuleList를 사용해서 아래와 같이 짜주면 된다.

model = nn.ModuleList([block,block])
x = torch.randn(1)
y = torch.randn(1)
for block in model:
    x, y = block(x,y)
print(x, y)